In [ ]:
# 필요한 라이브러리 임폴트

from inception_resnet_v1 import *

from functools import partial

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import numpy as np
# PIL -> PythonImageLibrary
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
# 사전 학습된 모델 불러와서 기본 모델 생성하기
# 'inceptionReNetV1()'의 경우 같은 경로에 저장된 inception_resnet_v1 파일 내부에 저장된 모델을 불러옴
base_model = InceptionResNetV1(weights_path='./facenet_keras_weights.h5',
                              input_shape=(224,224,3),
                              dropout_keep_prob=0.8)

In [ ]:
# 기본 모델 구조 확인하기

print(base_model.summary())

In [ ]:
# 기본 모델의 각 계층 인덱스와 이름 확인하기
# enumerate를 for문에 쓸 경우 인덱스와 담긴 값을 같이 가져옴
for i , layer in enumerate(base_model.layers):
    print(i, layer.name)
    
# 이렇게 모델의 계층 인덱스를 확인하는 이유는 전이학습을 위해 특성추출 부분과 
# 분류 부분의 경계선을 구분하여전이학습 모델을 만들기 위함
# Conv2d라는 부분을 통해 통해 연산하여 특성을 추출함(계층에 conv2d가 들어 있는건 모두 특성 추출)
# Concatenate 이후 Block8_6_Conv2d_1x1로 펴주면 성능이 향상된다는 논문이 있어서 이렇게 펴줌

In [ ]:
# trainable = False를 이용하여 기본 모델의 전 계층 동결하기
# 계층 동결시키는 이유? 학습을 시키지 않으면 모델의 성능을 개선하지 않고 그대로 사용함
# 계층을 동결시키므로 대신 학습 시간을 줄일 수 있음
# 즉 동결시키는 부분은 학습이 안되고 나머지 부분에서 학습을 시키는 거 자체가 전이학습임
for layer in base_model.layers:
    layer.trainable = False
    
# 결과 확인
print(base_model.summary())

In [ ]:
# 전이 학습 모델 정의 하기(분류 모델 부분만 전이학습 시킨 것)

classes = 50

x = base_model.get_layer(index=442).output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation = "relu",
         kernel_initializer = "he_normal",
         bias_initializer = "zeros")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(classes, activation = "softmax")(x)

my_model = Model(inputs = base_model.input, outputs=predictions)

# 결과 확인
print('base_model.get_layer(index=442).output의 모양 = ', base_model.get_layer(index=442).output.shape)
print("전이 학습 모델의 입력인 base_model.input의 모양 = ", base_model.input.shape)
print("전이 학습 모델의 출력인 predictions의 모양 = ", predictions.shape)

print(my_model.summary())

# 증식시키지 않고 학습하기

In [ ]:
# 학습용 ImageDataGenerator() 함수에 증식에 관한 어떤 조건도 부여하지 않기

# 1 뒤에 .를 찍는 이유는 정수 뒤에 .을 찍으면 정수가 실수로 바뀌므로 아래의 값이 정수가 아닌 실수로 바뀌므로
train_datagen1 = ImageDataGenerator(rescale=1./255)

# 검증용 ImageDataGenerator() 함수 설정
val_datagen1 = ImageDataGenerator(rescale = 1./255)

# 학습용 데이터와 검증용 데이터 경로 설정
print("현재 경로 : ", os.getcwd())
train_dir = os.getcwd() + "/img/train"
val_dir = os.getcwd() + "/img/val"

# flow_from_directory() 함수를 이용, ImgeDataGenerator()에 학습용 데이터, 테스트용 데이터 공급
# target_size = (HEIGHT, WIDTH) : 이미지 크기 설정
# keras 모델의 경우 높이를 먼저 써줘야함
HEIGHT = 224
WIDTH = 224

# 폴더명 자체가 class가 됨 따라서(class는 goeun/gongyu이 됨)
# flow_from_directory를 쓰면 저절로 폴더 이름을 label이라고 인식하도록 설계 되어 있음(부가기능)
# flow_from_directory는 데이터를 연결시켜주는 기능을 함
train_generator1 = train_datagen1.flow_from_directory(train_dir,
                                                     batch_size=20,
                                                     target_size=(HEIGHT,WIDTH),
                                                     shuffle=True,
                                                      # 클래스가 복수이고 다중 분류이므로 categorical를 활용함
                                                      # 분류 모델의 경우 binary 모델을 활용
                                                     class_mode="categorical")

val_generator1 = val_datagen1.flow_from_directory(val_dir,
                            batch_size=20,
                            target_size=(HEIGHT,WIDTH),
                            shuffle=True,
                            class_mode="categorical")

In [ ]:
# 학습 결과를 저장할 디렉토리 생성
checkpoint_dir = os.getcwd() + "/model"
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
# 과대 적합을 방지하기 위한 조기 종료 설정
early_stop = EarlyStopping(monitor = 'val_loss', verbose = 1, patience= 5)

# 최적의 학습 결과를 저장하기 위한 ModelCheckpoint 설정
# min의 의미는 손실값이 최소화될 때마다 저장하게 만듦
checkpoint = ModelCheckpoint(filepath=checkpoint_dir+"/"+"no_augment_weight.h5", monitor = "val_loss",
                            verbose=1, mode='min', save_best_only=True)

# 모델 컴파일
loss = "categorical_crossentropy"
optimizer = Adam(lr=0.0001)
my_model.compile(loss = loss, optimizer = optimizer, metrics=["acc"])

# 학습 진행
history = my_model.fit_generator(train_generator1,
                                 steps_per_epoch=len(train_generator1),
                                 epochs = 100,
                                 validation_data = val_generator1,
                                 validation_steps = len(val_generator1),
                                 callbacks = [early_stop, checkpoint]
                                )

In [ ]:
# 학습 결과 시각화
acc = history.history['acc']
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1,len(acc)+1)

plt.figure(figsize=(10,5))
# "bo"-> 파란색 점을 찍음 / "b"-> 파란색 선을 그려줌
plt.plot(epochs, acc, "bo", label= "Training acc")
plt.plot(epochs, val_acc, "b", label = "Validation_acc")
plt.plot(epochs, loss, "ro", label = "Training loss")
plt.plot(epochs, val_loss, "r", label = "Validation_loss")

plt.legend()

plt.show()

# 학습용 이미지를 증식시켜서 학습 시키기

In [ ]:
# ImageDataGenerator() 함수와 flow_from_directory() 함수를 사용

# 학습용은 증식해야함
train_datagen2 = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 15,
                                    width_shift_range = 0.15,
                                    height_shift_range = 0.15,
                                    # 찌그러 뜨리기(shear_range)
                                    shear_range = 0.15,
                                    zoom_range = 0.10,
                                    horizontal_flip = True,
                                    vertical_flip= True,
                                    # 어떻게 채울지 선택하는 것(fill_mode)
                                    fill_mode = "nearest"
                                   )
#검증용은 증식하면 안됌 검증에 쓰여야 하기 때문
val_datagen2 = ImageDataGenerator(rescale = 1./255)

# 데이터 경로 설정
train_dir = os.getcwd()+"/img/train"
val_dir = os.getcwd()+"/img/val"

# flow_from_directory() 함수 이용, 데이터 공급
train_generator2 = train_datagen2.flow_from_directory(train_dir,
                                                     batch_size=30,
                                                     target_size=(HEIGHT,WIDTH),
                                                     shuffle=True,
                                                     class_mode="categorical"
                                                     )

val_generator2 = val_datagen2.flow_from_directory(val_dir,
                                                  batch_size=10,
                                                  target_size=(HEIGHT,WIDTH),
                                                  shuffle=True,
                                                  class_mode="categorical"
                                                 )

In [ ]:
# 최적의 학습 결과를 저장(대신 변수명을 다르게 지정해야함)
checkpoint2 = ModelCheckpoint(filepath=checkpoint_dir+"/"+"augment_weight.h5", monitor = "val_loss",
                            verbose=1, mode='min', save_best_only=True)

history2 = my_model.fit_generator(train_generator2,
                                  steps_per_epoch=len(train_generator2),
                                  epochs = 100,
                                  validation_data = val_generator2,
                                  validation_steps = len(val_generator2),
                                  callbacks = [early_stop, checkpoint2]                             
                                 )

In [ ]:
# 학습 결과 시각화
acc = history2.history['acc']
val_acc = history2.history["val_acc"]
loss = history2.history["loss"]
val_loss = history2.history["val_loss"]

epochs = range(1,len(acc)+1)

plt.figure(figsize=(10,5))
# "bo"-> 파란색 점을 찍음 / "b"-> 파란색 선을 그려줌
plt.plot(epochs, acc, "bo", label= "Training acc")
plt.plot(epochs, val_acc, "b", label = "Validation_acc")
plt.plot(epochs, loss, "ro", label = "Training loss")
plt.plot(epochs, val_loss, "r", label = "Validation_loss")

plt.legend()

plt.show()

# 학습결과 확인하기

In [ ]:
# 모델 불러오기

import os
from tensorflow.keras.models import load_model

# 학습된 이미지 저장 경로 설정
checkpoint_dir = os.getcwd() + '/model/'

# 학습된 모델 불러오기
model1 = load_model(checkpoint_dir + 'no_augment_weight.h5')

# 결과 확인하기
print(model1.summary())

In [ ]:
# test 폴더에 있는 이미지를 이용해서 인식 / 분류 실행

# PIL -> PythonImageLibrary
from PIL import Image

# 이미지 사이즈 실행
HEIGHT = 224
WIDTH = 224

# 테스트 이미지 경로 설정
test_dir = os.getcwd() + "/img/test/"

# 시각화를 위한 설정
image_path = []


# os.listdir() 해당 폴더 안에 있는 파일들을 리스트화 하여 보내줌
for name in os.listdir(test_dir):
    # 각 이미지의 절대 경로 설정
    test_image_name = test_dir + name
    print(test_image_name)
    image_path.append(test_image_name)
    
# image_path : 테스트 이미지 각각의 절대 경로를 리스트로 저장
print("\n")
print(image_path)

for image in image_path:
    # Image.open() -> 이미지 불러오는 함수 <이 함수는 tuple로 전달하게 만들어짐>
    img = Image.open(image)
    print("\n")
    print("img 자료형 확인 = {}".format(type(img)))
    img = img.resize((HEIGHT, WIDTH))
    data = np.array(img)
    print("data 자료형 확인 = {}".format(type(data)))
    X = np.array(data)
    X = X.astype("float")/255 # 타입을 정수에서 실수로 바꾸어 0~1의 정규화를 진행
    X = X.reshape(-1, HEIGHT, WIDTH, 3)
    categories = ["baleut","bulgogi","burger","chik","cho","coco","dag","dang","don",
                "dongbaek","fish","galbi","go","gwang","han","hang","hong",
                "hwang","jamami","jin","jingal","kagawa","kao","kor","kum","mali",
                "mama","man","mok","moon","mul","nak","oct","pong","pyeon","red",
                "sakana","sinbul","sirae","snake","snoopy","sot","spa","sukju",
                "theflower","two","vivi","yasmaru","zerk","zuk"]
    pred = model1.predict(X)

    # [np.argmax()] 값의 최대값을 가져오는 함수
    result = [np.argmax(value) for value in pred]
    print('New image prediction : ',categories[result[0]])
    print("accuracy : {}".format(max(pred[0])))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

# 이미지 증식 확인하기

In [ ]:
# 모델 불러오기

import os
from tensorflow.keras.models import load_model

# 학습된 이미지 저장 경로 설정
checkpoint_dir = os.getcwd() + '/model/'

# 학습된 모델 불러오기
model2 = load_model(checkpoint_dir + 'augment_weight.h5')

# 결과 확인하기
print(model2.summary())

In [ ]:
# test 폴더에 있는 이미지를 이용해서 인식 / 분류 실행

# PIL -> PythonImageLibrary
from PIL import Image

# 이미지 사이즈 실행
HEIGHT = 224
WIDTH = 224

# 테스트 이미지 경로 설정
test_dir = os.getcwd() + "/img/test/"

# 시각화를 위한 설정
image_path = []


# os.listdir() 해당 폴더 안에 있는 파일들을 리스트화 하여 보내줌
for name in os.listdir(test_dir):
    # 각 이미지의 절대 경로 설정
    test_image_name = test_dir + name
    print(test_image_name)
    image_path.append(test_image_name)
    
# image_path : 테스트 이미지 각각의 절대 경로를 리스트로 저장
print("\n")
print(image_path)

for image in image_path:
    # Image.open() -> 이미지 불러오는 함수 <이 함수는 tuple로 전달하게 만들어짐>
    img = Image.open(image)
    print("\n")
    print("img 자료형 확인 = {}".format(type(img)))
    img = img.resize((HEIGHT, WIDTH))
    data = np.array(img)
    print("data 자료형 확인 = {}".format(type(data)))
    X = np.array(data)
    X = X.astype("float")/255 # 타입을 정수에서 실수로 바꾸어 0~1의 정규화를 진행
    X = X.reshape(1, HEIGHT, WIDTH, 3)
    categories = ["baleut","bulgogi","burger","chik","cho","coco","dag","dang","don",
                "dongbaek","fish","galbi","go","gwang","han","hang","hong",
                "hwang","jamami","jin","jingal","kagawa","kao","kor","kum","mali",
                "mama","man","mok","moon","mul","nak","oct","pong","pyeon","red",
                "sakana","sinbul","sirae","snake","snoopy","sot","spa","sukju",
                "theflower","two","vivi","yasmaru","zerk","zuk"]
    pred = model2.predict(X)
    # [np.argmax()] 값의 최대값을 가져오는 함수
    result = [np.argmax(value) for value in pred]
    print('New image prediction : ',categories[result[0]])
#     print("accuracy : {}".format(max(pred[0][0],pred[0][1])))
    print("accuracy : {}".format(max(pred[0])))
    plt.axis('off')
    plt.imshow(img)
    plt.show()

# 모델 안드로이드로 변환

In [ ]:
# from tensorflow import keras
# export_path = "./pb"
# model2.save(export_path, save_format='tf')

# Pd 변환

In [ ]:
# import tensorflow as tf

# saved_model_dir = './pb'
# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# tflite_model = converter.convert()
# open('./tf/converted_model.tflite', 'wb').write(tflite_model)

# 서버연동 위한 코드 정리

In [ ]:
import urllib.request
import time
from PIL import Image

# 다운받을 이미지 url
url = "https://firebasestorage.googleapis.com/v0/b/finfooproject.appspot.com/o/%EC%BD%94%EC%BD%94%EC%8A%A4%ED%86%A4%20(11).jpg?alt=media&token=f23be5a6-5aa4-42eb-b501-00134a9385ec"

# time check
start = time.time()

# 이미지 요청 및 다운로드
urllib.request.urlretrieve(url, "test.jpg")

# 이미지 다운로드 시간 체크
print(time.time() - start)

# 저장 된 이미지 확인
img = Image.open("test.jpg")
img = img.resize((HEIGHT, WIDTH))
data = np.array(img)
print("data 자료형 확인 = {}".format(type(data)))
X = np.array(data)
X = X.astype("float")/255 # 타입을 정수에서 실수로 바꾸어 0~1의 정규화를 진행
X = X.reshape(1, HEIGHT, WIDTH, 3)
categories = ["baleut","bulgogi","burger","chik","cho","coco","dag","dang","don",
                "dongbaek","fish","galbi","go","gwang","han","hang","hong",
                "hwang","jamami","jin","jingal","kagawa","kao","kor","kum","mali",
                "mama","man","mok","moon","mul","nak","oct","pong","pyeon","red",
                "sakana","sinbul","sirae","snake","snoopy","sot","spa","sukju",
                "theflower","two","vivi","yasmaru","zerk","zuk"]
pred = model2.predict(X)
# [np.argmax()] 값의 최대값을 가져오는 함수
result = [np.argmax(value) for value in pred]
print('New image prediction : ',categories[result[0]])
#print("accuracy : {}".format(max(pred[0][0],pred[0][1])))
print("accuracy : {}".format(max(pred[0])))